# Deploying a Dask Hub

All this material is taken from the following docs:
- https://docs.dask.org/en/latest/setup/kubernetes-helm.html
- https://zero-to-jupyterhub.readthedocs.io/en/latest/kubernetes/setup-kubernetes.html
- https://zero-to-jupyterhub.readthedocs.io/en/latest/kubernetes/setup-helm.html

## Creating a Kubernetes Cluster

First, you need to enable the Kubernetes API if not already done:
1. Go to console.cloud.google.com
2. Select the Kubernetes Engine in the menu: https://console.cloud.google.com/marketplace/product/google/container.googleapis.com
3. Enable the API if not already done.

Then you'll need a terminal with __gcloud__ and __kubectl__. The simplest is just to use the Google Cloud Shell from console.cloud.google.com. If you prefer, you can follow the links above to find how to install everything on your computer.

Ask Google Cloud to create a managed Kubernetes cluster and a default node pool to get nodes from:

```
gcloud container clusters create \
  --machine-type n1-standard-4 \
  --enable-autoscaling \
  --min-nodes 1 \
  --max-nodes 10 \
  --num-nodes 1 \
  --zone europe-west1-b \
  --cluster-version 1.21 \
  dask-hub-k8s
```

Yhis will take a few minutes (maybe 2 or 3).

You can then test if the cluster is running:
```
kubectl get node
```

Then get permissions to perform all administrative actions needed. 

**⚠️Don't forget to replace your email below.⚠️**

```
kubectl create clusterrolebinding cluster-admin-binding \
  --clusterrole=cluster-admin \
  --user=<GOOGLE-EMAIL-ACCOUNT>
```

## Setting up Helm

From your Google Cloud Shell or terminal:

```
curl https://raw.githubusercontent.com/helm/helm/master/scripts/get-helm-3 | bash
helm list
```

should return:
```
NAME    NAMESPACE       REVISION        UPDATED STATUS  CHART   APP VERSION
```


## Helm install a Dask Hub

Default Daskhub configuration uses dask-gateway, which is here to handle multiple users with fine grain authorisations. We don't need this, and it iss a little more complicated setup than what we'll do.

Instead, we'll deploy a Daskhub with dask-kubernetes, which assumes some authorisations inside the Pods of the Kubernetes cluster (potential security leak), but is more straightforward for our usage.

Verify that you’ve set up a Kubernetes cluster and added Dask’s helm charts:

```
helm repo add dask https://helm.dask.org/
helm repo update
```

Generate token to configure Jupyterhub:

```
openssl rand -hex 32  # generate token-1
```

Create the file below (for example using vim or cloud shell editor) and substitute the `<token-1>` value.

```yaml
# file: daskhub-config.yaml
jupyterhub:
  proxy:
    secretToken: "<token-1>"
  scheduling:
    podPriority:
      enabled: true
    userPlaceholder:
      # Specify three dummy user pods will be used as placeholders
      replicas: 1
    userScheduler:
      enabled: true
  singleuser:
    serviceAccountName: daskkubernetes
    image:
      name: guillaumeeb/pangeo-ml-notebook # Image to use for singleuser environment. Must include dask-kubernetes.
      tag: 2021.11.14

dask-gateway:
  enabled: false
  gateway:
    auth:
      type: simple
      simple:
        password: "unused"

dask-kubernetes:
  enabled: true
```

Now we just install Dask Hub:
```
helm upgrade --wait --install --render-subchart-notes \
    --namespace daskhub \
    --create-namespace \
    dhub dask/daskhub \
    --values=daskhub-config.yaml
```

This will again take a few minutes.

## Check install and go to Jupyter!

To get the public IP of your hub deployment:
```
kubectl --namespace=daskhub get service proxy-public
```

Get the external IP, and open it in your browser. You should be able to login with any username/password

## Ensure Dask is working, and K8S mecanisms too!

### Create a dask-kubernetes cluster

Create a yaml file within the Jupyterhub interface:

```yaml
# worker-spec.yaml

kind: Pod
metadata:
  labels:
    foo: bar
spec:
  restartPolicy: Never
  containers:
  - image: guillaumeeb/pangeo-ml-notebook:2021.11.14
    imagePullPolicy: IfNotPresent
    args: [dask-worker, --nthreads, '2', --no-dashboard, --memory-limit, 6GB, --death-timeout, '60']
    name: dask
    env:
      - name: EXTRA_PIP_PACKAGES
        value: xgboost
    resources:
      limits:
        cpu: "2"
        memory: 6G
      requests:
        cpu: "1.7"
        memory: 6G

```

Just open a notebook in your newly created Dask enabled hub, and try to copy and past the following cells:

Set some config to ease usage.

In [ ]:
import dask
import dask.distributed  # populate config with distributed defaults
import dask_kubernetes

dask.config.set({"kubernetes.worker-template-path": "worker-spec.yaml"})
dask.config.set({"distributed.dashboard.link": "{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status"})


Create a cluster object.

In [ ]:
from dask_kubernetes import KubeCluster

cluster = KubeCluster(deploy_mode='local') # Scheduler is started in the notebook process
cluster

This should display a fancy widget. You can open the Dask Dashboard from here. 

Now scale the cluster to get Dask-workers and connect to it.

In [ ]:
cluster.scale(20)

In [ ]:
from distributed import Client

client = Client(cluster)
client

#### _What's happening in your K8S cluster after some seconds/minutes?_

### Launch some computation, what about Pi?

We'll use Dask array, a Numpy extension for this:

In [ ]:
import dask.array as da

sample = 10_000_000_000  # <- this is huge!
xxyy = da.random.uniform(-1, 1, size=(2, sample))
norm = da.linalg.norm(xxyy, axis=0)
summ = da.sum(norm <= 1)
insiders = summ.compute()
pi = 4 * insiders / sample
print("pi ~= {}".format(pi))

#### _How many workers did you get? Why?_

Now just close the cluster.

In [ ]:
cluster.close()

#### _What happens after a few minutes?_

## Access GCS Data

In [ ]:
import pandas as pd
df = pd.read_csv('gs://obd-dask/train.csv', nrows = 1_000_000)
df

# Backup plan: Dask Hub with dask-gateway

Generate tokens to configure Jupyterhub and Dask-gateway:

```
openssl rand -hex 32  # generate token-1
openssl rand -hex 32  # generate token-2
```

Create the file below (for example using vim or cloud shell editor) and substitute those two values for `<token-1>` and `<token-2>`.

**⚠️ \<token-2\> must be replaced in two places!⚠️**
    
```
# file: secrets.yaml
jupyterhub:
  proxy:
    secretToken: "<token-1>"
  hub:
    services:
      dask-gateway:
        apiToken: "<token-2>"
  scheduling:
    podPriority:
      enabled: true
    userPlaceholder:
      # Specify three dummy user pods will be used as placeholders
      replicas: 1
    userScheduler:
      enabled: true
  singleuser:
    image:
      name: pangeo/ml-notebook  # Image to use for singleuser environment. Must include dask-gateyway.
      tag: 2021.11.09

dask-gateway:
  gateway:
    auth:
      jupyterhub:
        apiToken: "<token-2>"
    extraConfig:
      optionHandler: |
        from dask_gateway_server.options import Options, Integer, Float, String

        def options_handler(options):
          if ":" not in options.image:
            raise ValueError("When specifying an image you must also provide a tag")
          return {
            "worker_cores": options.worker_cores,
            "worker_memory": int(options.worker_memory * 2 ** 30),
            "image": options.image,
          }

        c.Backend.cluster_options = Options(
          Integer("worker_cores", default=1, min=1, max=4, label="Worker Cores"),
          Float("worker_memory", default=1, min=1, max=8, label="Worker Memory (GiB)"),
          String("image", default="pangeo/ml-notebook:2021.11.09", label="Image"),
          handler=options_handler,
        )
```

# If you're behind a corporate proxy (like me)

Try to enable SSL (https).
That does not work with dask-gateway.

Generate a self signed certificate, and put it inside Kubernetes secrets:

```bash
openssl req -newkey rsa:2048 -nodes -keyout domain.key -x509 -days 365 -out domain.crt
kubectl --namespace daskhub create secret tls domain-tls --key="domain.key" --cert="domain.crt"
```

Update the yaml configuration file, in the jupyterhub.proxy section, enabling https and pointing to your certificate:
```yaml
jupyterhub:
  proxy:
    https:
      enabled: true
      #      hosts:
      #  - 34.140.104.238
      type: secret
      secret:
        name: domain-tls
```

Refresh helm deployment:

```bash
helm upgrade --cleanup-on-fail --render-subchart-notes \
    --namespace daskhub \
    --create-namespace \
    dhub dask/daskhub \
    --values=daskhub-config.yaml
```

### Connect to Dask gateway

In [ ]:
from dask_gateway import Gateway
# Use values stored in your local configuration (recommended)
gateway = Gateway()

Is there any existing Dask cluster in there?

In [ ]:
gateway.list_clusters()

### Launch a Dask cluster

In [ ]:
cluster = gateway.new_cluster(worker_cores=1, worker_memory=3.0)
cluster

This should display a fancy widget. You can open the Dask Dashboard from here. 

Now connect to the cluster, and scale it to get Dask workers.

In [ ]:
client = cluster.get_client()
cluster.scale(20)